# Random Forest Implemented from Scratch in Python OOP

In [12]:
import numpy as np
from sklearn import datasets
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [11]:
class Node: # a node is one point in the tree
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None): # for a question node or "internal" node
        self.feature = feature # column aka which question to ask at the crossroads
        self.threshold = threshold # cutoff value for the question
        self.left = left # pointers to the next notes (what to do for "yes" and "no" of questions)
        self.right = right
        self.value = value

    def is_leaf_node(self): # for a leaf node
        return self.value is not None # store the predicted label

# The model
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split  = min_samples_split # don't split if very few exmaples are left
        self.max_depth = max_depth # don't let the tree get too tall
        self.n_features = n_features # how many features to consider at each split, if none use them all
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features) # how many features does the data have
        self.root = self._grow_tree(X, y) # call to build the whole tree and stores the top as the root

    # Building the tree
    def _grow_tree(self, X, y, depth=0): # recursive function
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # Check the stopping criteria
        # if reached max depth, if all examples have the same label (already pure), if there are too few examples
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y) # make a leaf whose value is the most common label in y
            return Node(value=leaf_value)

        # Find the best split, aka find the best question
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False) # randomly pick some candidate features to try
        # choose the feature and threshold combination with the highest gain
        best_feature, best_thresh = self._best_split(X, y, feat_idxs) # for each candidate feature, try possible thresholds and compute how good each split is using IG

        # Create child nodes, aka split the data into left and right (according to threshold value)
        left_idxs, right_idxs = self._split(X[:,best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1) # recursively build the left subtree from left data and right subtree from right data 
        right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right) # node that stores the chosen feature, threshold, and links to its left/right children

    # Look at the labels in this group and return the one that appears the most (aka the majority)
    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    # Try candidate features and many different thresholds for each
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain for each trial
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = thr
        return split_idx, split_thresh
    
    # Measure how much a split reduces impurity
    def _information_gain(self, y, X_column, threshold):
        # parent entropy (before splitting)
        parent_entropy = self._entropy(y)

        # create children, aka split indexes left and right groups using threshold
        left_idxs, right_idxs = self._split(X_column, threshold)
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted average of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs]) # compute ledt and right entropy
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain # Bigger gain means the split made the groups more pure ⇒ better question
    
    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y) # divide occurrance counts by total values
        return -np.sum([p * np.log(p) for p in ps if p > 0])
    
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs # returns the indexes of rows that go left (≤ threshold) and right (> threshold)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X]) # for each new row, call traverse tree
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node(): # if the current node is a leaf, return its stored label
            return node.value

        if x[node.feature] <= node.threshold: # otherwise ask the node's question and assign it to left or right according to tresh
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)




In [28]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                         max_depth=self.max_depth,
                         n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X,y)
            tree.fit(X_sample,y_sample)                
            self.trees.append(tree)
        
    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True) # random sample can be selected more than once
        return X[idxs], y[idxs]
    
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions


In [29]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [31]:
model = RandomForest(n_trees=20)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

acc = accuracy(y_test, predictions)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.9649
